In [1]:
import os
import json
from dotenv import load_dotenv
from lib import fetch_cubes_descriptions, fetch_cube_sample, fetch_dimensions_triplets
from openai._client import OpenAI

load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [2]:
client = OpenAI(api_key = OPENAI_API_KEY)

In [3]:
cubes = fetch_cubes_descriptions()
question = f"average of emission of Methane for transport between years 2007 and 2005"

cubes_description = """
Given following data cubes with its labels and description:
{cubes}""".format(cubes=cubes)

human_template = "Select a cube, which would be best to answer following question: {question}. Return cube ID.".format(question=question)

In [4]:
for i in range(0,1):
  response = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    response_format={ "type": "json_object" },
    temperature= 0.5,
    messages=[
      {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
      {"role": "system", "content": cubes_description},
      {"role": "user", "content": human_template}
    ]
  )
  content = json.loads(response.choices[0].message.content)
  print(content)


{'cubeId': 'https://environment.ld.admin.ch/foen/ubd000502/4'}


In [5]:
selected_cube = "<" +list(content.values())[0] + ">"
cube_and_sample = fetch_cube_sample( selected_cube )
print(cube_and_sample)

@prefix schema: <http://schema.org/> .
@prefix qudt: <http://qudt.org/schema/qudt/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix cube: <https://cube.link/> .

<https://environment.ld.admin.ch/foen/ubd000502/4> a cube:Cube ;
   cube:observationSet <https://environment.ld.admin.ch/foen/ubd000502/4/observation/> .
<https://environment.ld.admin.ch/foen/ubd000502/4/observation/> a cube:observationSet ;
   cube:observation <https://environment.ld.admin.ch/foen/ubd000502/4/observation/32/CH4/2009> .
<https://environment.ld.admin.ch/foen/ubd000502/4/observation/32/CH4/2009> a cube:Observation ;
   cube:observedBy <https://ld.admin.ch/office/VII.1.7> ;
   <https://environment.ld.admin.ch/foen/ubd000502/sektorid> <https://environment.ld.admin.ch/vocabulary/ghg_emission_sectors_co2_ordinance/32> ;
   <https://environment.ld.admin.ch/foen/ubd000502/gas> <https://ld.admin.ch

In [6]:
dimensions_triplets = fetch_dimensions_triplets(selected_cube)

print(dimensions_triplets)

@prefix schema: <http://schema.org/> .
@prefix qudt: <http://qudt.org/schema/qudt/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix cube: <https://cube.link/> .

<https://environment.ld.admin.ch/vocabulary/ghg_emission_sectors_co2_ordinance/1> schema:name "Switzerland’s greenhouse gas emissions"@en .
<https://ld.admin.ch/cube/dimension/pol01air/pol11> schema:name "Methane"@en .
<https://environment.ld.admin.ch/vocabulary/ghg_emission_sectors_co2_ordinance/2> schema:name "International aviation (not included in total)"@en .
<https://ld.admin.ch/cube/dimension/pol01air/pol9> schema:name "Carbon dioxide"@en .
<https://environment.ld.admin.ch/vocabulary/ghg_emission_sectors_co2_ordinance/3> schema:name "Balance of land use (not included in total)"@en .
<https://environment.ld.admin.ch/vocabulary/ghg_emission_sectors_co2_ordinance/11> schema:name "Buildings"@en .
<https

In [8]:
content

{'error': 'The provided SPARQL query is invalid. Please correct the query and try again.'}

In [50]:
sample_description = """
Given cube and its sample observation:
{cube_and_sample}
""".format(cube_and_sample = cube_and_sample)

structure_description = """
Dimensions labels:
{dimensions_triplets}
""".format(dimensions_triplets=dimensions_triplets)

query_template = """
Sample query:
    PREFIX cube: <https://cube.link/>
    PREFIX schema: <http://schema.org/>
    PREFIX qudt: <http://qudt.org/schema/qudt/>
    PREFIX sh: <http://www.w3.org/ns/shacl#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

    SELECT *
    WHERE {{
    {cube} a cube:Cube;
        cube:observationSet ?observationSet.

    ?observationSet a cube:ObservationSet;
        cube:observation ?observation.

    ?observation a cube:Observation.
    }}
""".format(cube=selected_cube)


human_template = f"Modify sample query to answer following question: \n{question} using cube: {selected_cube}"


response = client.chat.completions.create(
  model="gpt-3.5-turbo-1106",
  response_format={ "type": "json_object" },
  temperature=0.15,
  top_p=0.1,
  messages=[
    {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
    {"role": "system", "content": sample_description},
    {"role": "system", "content": structure_description},
    {"role": "system", "content": query_template},
    {"role": "user", "content": human_template}
  ]
)
content = json.loads(response.choices[0].message.content)
print(content)

{'query': 'PREFIX cube: <https://cube.link/> PREFIX schema: <http://schema.org/> PREFIX qudt: <http://qudt.org/schema/qudt/> PREFIX sh: <http://www.w3.org/ns/shacl#> PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> PREFIX xsd: <http://www.w3.org/2001/XMLSchema#> SELECT (AVG(?value) AS ?averageEmission) WHERE { <https://environment.ld.admin.ch/foen/ubd000502/4> a cube:Cube; cube:observationSet ?observationSet. ?observationSet a cube:ObservationSet; cube:observation ?observation. ?observation a cube:Observation; <https://environment.ld.admin.ch/foen/ubd000502/gas> <https://ld.admin.ch/cube/dimension/pol01air/pol11>; <https://environment.ld.admin.ch/foen/ubd000502/jahr> ?year; <https://environment.ld.admin.ch/foen/ubd000502/werte> ?value. FILTER (?year >= "2005"^^xsd:gYear && ?year <= "2007"^^xsd:gYear) }'}
